## Move to python file

In [1]:
def create_resolved_ibcontract(stock_to_trade):
    # Create the contract object
    ibcontract = ib.IBcontract()
    ibcontract.secType = "STK"
    ibcontract.symbol = stock_to_trade
    ibcontract.currency = "USD"
    resolved_ibcontract = app.resolve_ib_contract(ibcontract)
    print(resolved_ibcontract)

    # TODO: Investigate that we are getting the right stock out of all of the options
    return resolved_ibcontract

In [2]:
def get_latest_price(resolved_ibcontract):
    
    # Get the last price
    tickerid = app.start_getting_IB_market_data(resolved_ibcontract)
    time.sleep(1)
    market_data1 = app.stop_getting_IB_market_data(tickerid)
    market_data1_as_df = market_data1.as_pdDataFrame()
    display(market_data1_as_df.resample("1S").last())
    some_quotes = market_data1_as_df.resample("1S").last()[["last_trade_price"]]
    current_price = some_quotes.iloc[0][0]
    print(current_price)
    
    return current_price
    # TODO: Skip this stock if price comes back as nan

## Reading my Chinese Alipay file. Not related to algo

In [ ]:
from chardet import detect
encoding = lambda x: detect(x)['encoding']

In [ ]:
f = open('/Users/chingaling/Desktop/alipay_record_20181112_1301_1.csv','r')

In [ ]:
type(f)

In [ ]:
for line in f:
    print encoding(line)
f.close()

In [ ]:
  n_line=unicode(line,encoding(line),errors='ignore')
  print n_line
  print n_line.encode('utf8')

## Beginning of algo playground

In [3]:
import sys
sys.path.insert(0, '/Users/chingaling/Documents/Algo Trading/BroCodePC')
import IB_API_tutorial as ib
import main as m
import time
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import math
import datetime
from itertools import cycle

In [71]:
from importlib import reload
reload (ib)

<module 'IB_API_tutorial' from '/Users/chingaling/Documents/Algo Trading/BroCodePC/IB_API_tutorial.py'>

In [ ]:
sys.path.insert(0, '/Users/chingaling/Documents/Algo Trading/RobCarver')
import L3StreamData

## Create some global variables and open connection

In [68]:
app.disconnect()

In [4]:
port = 7496
ip = "127.0.0.1"
clientID = 1
buyFactor = .99
SellFactor = 1.01
rebalance_interval = 10
maxBuyOrdersAtOnce = 30
minInvSize = 1000
stock_list = []
maxCandidates=100
leastPrice=3.00
mostPrice=20.00
fireSalePrice = leastPrice
fireSaleAge = 6

# Local file addresses
csvAddress_stocks_to_trade = "/Users/chingaling/Documents/Algo Trading/BroCodePC/stocks_to_trade.csv"
csvAddress_orders = "/Users/chingaling/Documents/Algo Trading/BroCodePC/orders.csv"
pklAdd_orders = "/Users/chingaling/Documents/Algo Trading/BroCodePC/orders.pkl"
pklAdd_exec = "/Users/chingaling/Documents/Algo Trading/BroCodePC/executions.pkl"

In [72]:
# Test connection
#clientID = 2
app = ib.TestApp(ip, port, clientID)
current_time = app.speaking_clock()
print(current_time)

Getting the time from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfuture
IB error id -1 errorcode 2104 string Market data farm connection is OK:eufarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:njfarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm.us
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ilhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:euhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:fundfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds
1542564953


In [ ]:
# Contract details to edit
ibcontract = ib.IBcontract()
ibcontract.secType = "STK"
ibcontract.symbol="TBIO"
ibcontract.currency = "USD"
resolved_ibcontract = app.resolve_ib_contract(ibcontract)
resolved_ibcontract

In [ ]:
historic_data = app.get_IB_historical_data(resolved_ibcontract)
historic_data

In [ ]:
PH = app.get_IB_historical_data(resolved_ibcontract, durationStr='20 D')

In [ ]:
PH_df = pd.DataFrame(PH, columns=['date', 'open', 'high', 'low', 'close', 'volume'])

In [ ]:
PH_df.close.mean()

In [ ]:
print(historic_data[-5:-1])

## Making buy orders

In [ ]:
def submit_buy_order(stock_to_trade):
    #if stock_to_trade in positions:
        #TODO: Check if this is actually a criteria in the Quantopian model
        #pass 
    resolved_ibcontract = create_resolved_ibcontract(stock_to_trade)
    PH = app.get_IB_historical_data(resolved_ibcontract, durationStr='20 D')
    PH_df = pd.DataFrame(PH, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    PH_Avg = float(PH_df.close.mean())
    current_price = float(get_latest_price(resolved_ibcontract))
    if np.isnan(current_price):
        pass # probably best to wait until nan goes away
    else:
        if current_price > float(1.25*PH_Avg):
            buyPrice=float(current_price)
        else:
            buyPrice=float(current_price*buyFactor)
        buyPrice=float(ib.make_div_by_05(buyPrice, buy=True))
        #StockShares = int(WeightThisBuyOrder*cash/buyPrice)
        StockShares = int(minInvSize/buyPrice)
        print("Buying {}, \tshares: {}, \tprice: {}".format(stock_to_trade, StockShares, buyPrice))

        # Submit the buy order and log the order
        orderid = app.submit_order(resolved_ibcontract=resolved_ibcontract, action="BUY", shares=StockShares, price=buyPrice, order_type="LMT", orderid=None, tif="GTC")
        order_pd = app.add_order_entry(orderid, pklAdd_orders=pklAdd_orders, csvAddress_orders=csvAddress_orders)

In [ ]:
def rebalance_buy_orders():
    WeightThisBuyOrder=float(1.00/maxBuyOrdersAtOnce)
    for ThisBuyOrder in range(maxBuyOrdersAtOnce):
        stock_to_trade = next(nextStock)
        submit_buy_order(stock_to_trade)
        

In [ ]:
# Contract details to edit
stock_to_trade = next(nextStock)
print("Trading this stock: {}".format(stock_to_trade))

# TODO: Check to see if this stock is already on the order list, or within the portfolio. Or if it's been traded already today (like sold). If so, skip to the next one.

In [ ]:
submit_buy_order(stock_to_trade)

In [ ]:
rebalance_buy_orders()

In [ ]:
# Load list of stocks to trade and create cycle object
stockNames = list(pd.read_csv(csvAddress_stocks_to_trade).Stocks)
nextStock = cycle(stockNames)
stockNames

In [ ]:
resolved_ibcontract = create_resolved_ibcontract(stock_to_trade)

In [ ]:
current_price = get_latest_price(resolved_ibcontract)

In [ ]:
# Find the price to buy and amount
buy_price = current_price * buyFactor
buy_price=float(ib.make_div_by_05(buy_price, buy=True))
print("buy_price: {}".format(buy_price))
buy_amount = math.floor(minInvSize/buy_price)
print("buy_amount: {}".format(buy_amount))

In [ ]:
# Submit the buy order and log the order
orderid = app.submit_order(resolved_ibcontract=resolved_ibcontract, action="BUY", shares=buy_amount, price=buy_price, order_type="LMT", orderid=None, tif="GTC")
order_pd = app.add_order_entry(orderid, pklAdd_orders=pklAdd_orders, csvAddress_orders=csvAddress_orders)

## Cancelling buy orders

In [ ]:
# Cancel all open buy orders
app.cancel_open_buy_orders()

In [ ]:
abs(order_pd.time_added[0] - datetime.datetime.now()) > datetime.timedelta(minutes=10)

In [ ]:
a = pd.read_pickle(pklAdd_orders)

In [ ]:
a

## Submitting sell orders

In [ ]:
def connect(self):
    self.app = ib.TestApp(ip, port, clientID)

In [ ]:
def main_trading():
    # Here goes all of the code to run throughout the trading day

    #Before trading day starts, load up list of stock candidates
    self.stock_list = generate_stock_shortlist()

    #TODO: Every rebalance_interval minutes, rebalance the portfolio
    while (current_time < end_trading_day):
        
        # Connect to the IB API for every rebalancing
        app = ib.TestApp("127.0.0.1", port, 1)
        rebalance(buyFactor=buyFactor, SellFactor=SellFactor)
        
        # At the end of the day, disconnect from the IB API app
        app.disconnect()
        
        # Wait for the rebalance_interval time (in minutes)
        time.wait(rebalance_interval * 60)

In [ ]:
def rebalance(buyFactor=.99, SellFactor=1.01):
    # Re-balancing method from 30_stock algo every 10 minutes. Buy order cycles through list of 100 stock selected each day.

    # Get outstanding buying power from account
    cash=get_buying_power()

    # Close out all outstanding buy orders that were not fulfilled. Should be most of them
    cancel_open_buy_orders()

    # Get list of current positions
    positions = get_current_positions()

    # Create new sell orders as per algo strategy
    rebalance_all_sell_orders()

    # Submit new buy orders
    if cash > minInvSize:
        rebalance_buy_orders()

In [ ]:
# Retrieve information from IB server
def get_buying_power(self):
    #TODO: Retrieve remaining buying power from account
    accounting_values = app.get_accounting_values(accountName)
    
    pass

In [ ]:
# Cancel all outstanding buy orders
open_orders = self.get_open_orders_pd()
for _, order in open_orders.iterrows():
    if order.order.action == "BUY":
        print("Cancelling this order: {}".format(order))
        print(order.order.orderId)
        self.cancel_order(order.order.orderId)

In [73]:
open_orders = app.get_open_orders_pd()
open_orders

,avgFillPrice,clientId,filled,id,lastFillPrice,mktCapPrice,orderstate,parentId,permid,status,...,tif,totalQuantity,trailStopPrice,trailingPercent,transmit,triggerMethod,triggerPrice,volatility,volatilityType,whatIf
0,0.0,1,0.0,18,0.0,0.0,<ibapi.order_state.OrderState object at 0x1086...,0,575567031,PreSubmitted,...,GTC,90.0,5.35,1.797693e+308,True,0,1.797693e+308,1.797693e+308,0,False


In [75]:
open_orders.

Index(['avgFillPrice', 'clientId', 'filled', 'id', 'lastFillPrice',
       'mktCapPrice', 'orderstate', 'parentId', 'permid', 'status',
       ...
       'tif', 'totalQuantity', 'trailStopPrice', 'trailingPercent', 'transmit',
       'triggerMethod', 'triggerPrice', 'volatility', 'volatilityType',
       'whatIf'],
      dtype='object', length=154)

In [ ]:
for _, order in open_orders.iterrows():
    if order.

In [ ]:
def get_open_sell_orders(self):
    #TODO: Retrieve list of all open sell orders:

    pass

In [ ]:
def generate_stock_shortlist(self):

    # Return list of stocks to trade for the day
    csvAddress = "/Users/chingaling/Documents/Algo Trading/BroCodePC/stocks_to_trade.csv"
    self.stock_list = list(pd.read_csv(csvAddress).Stocks)
    self.nextStock = cycle(stockNames)

In [ ]:
# Logging order entry
def add_order_entry(orderid):
    # Add a new log entry (somewhere, either CSV, cloud database, etc.) keeping track of all order detail
    
    # Get open orders
    open_orders = app.get_open_orders()
    order_attributes = ['contract','order','orderstate','status',
                 'filled', 'avgFillPrice', 'permid',
                 'parentId', 'lastFillPrice', 'clientId', 'whyHeld',
                'mktCapPrice']
    attributes = {}
    order_pd_base = pd.read_pickle(pklAdd_orders)
    if orderid in open_orders:
        orderInformation = open_orders[orderid]
        for attr in order_attributes:
            attributes[attr] = getattr(orderInformation, attr)
        order_pd_new = pd.DataFrame(attributes, index=[0])
        order_pd_new = order_pd_new.assign(time_added=datetime.datetime.now())
        columns = order_pd_new.columns
        order_pd = pd.concat([order_pd_base, order_pd_new])
        order_pd = order_pd[columns]
        
        # Save latest stData to pickle file and csv locally
        order_pd.to_pickle(pklAdd_orders)
        order_pd.to_csv(csvAddress_orders)
    else:
        print("Orderid {} not in open_orders".format(orderid))
        order_pd = None
    return order_pd

In [ ]:
def submit_order(resolved_ibcontract, action, shares, price, order_type, orderid, tif='GTC'):
    """
        Submits an order to IB API.

    Args:
        resolved_ibcontract: IBContract, must be fully resolved
        action: str, options: 'BUY', 'SELL'
        shares: int
        price: double
        order_type: str, options: 'MKT', 'LMT', etc.
        orderid: int, identifies which orderid to assign
        tif (optional): Time-in-force. Defaults to 'GTC'.

    Returns:
        Returns the orderid used for this order

    Raises:
        None
    """
    
    # Submit order instructions to IB server
    currOrder = ib.Order()
    currOrder.action = action
    currOrder.orderType = order_type
    currOrder.totalQuantity = shares
    currOrder.lmtPrice = price
    currOrder.tif = tif
    currOrder.transmit = True
    print("App is connected?: {}".format(app.isConnected()))
    currOrderId = app.place_new_IB_order(resolved_ibcontract, currOrder, orderid=orderid)
    print("Placed order, orderid is %d" % currOrderId)
    
    return currOrderId

## Getting executions and commission

In [ ]:
# Building up sell order

orderid = increment_order_no(orderid)
submit_order(resolved_ibcontract=resolved_ibcontract, action="BUY", shares=1, price=150, order_type="LMT", orderid=orderid, tif="GTC")
add_order_entry(orderid)

In [ ]:
symbols = {}
for index, row in order_history.iterrows():
    symbols[index] = row.contract.symbol
order_history = pd.concat([order_history, pd.Series(symbols, name='symbol')], axis=1)

In [ ]:
for symbol in positions.symbol:
    if symbol in order_history.contract:
        print (position)

In [ ]:
positions.contract[0]
order_history.contract

In [ ]:
positions = app.get_current_positions()

In [26]:
def rebalance_sell_order(position_series):

    # Receive positions Series from output of get_current_positions()
    
    StockShares = position_series.position
    
    # We have to re-resolve the contract based on the quote. Some risk in not identifying the same security. For some reason the saved Contract object in our positions DataFrame does not square up exactly with their resolved contract.
    resolved_ibcontract = create_resolved_ibcontract(position_series.symbol)
    CurrPrice = get_latest_price(resolved_ibcontract)
    
    CostBasis = float(position_series.avg_cost)
    SellPrice = float(ib.make_div_by_05(CostBasis*SellFactor, buy=False))
    
    stock = resolved_ibcontract.symbol
    today = datetime.datetime.now()

    # If the position is held for less than 2 business days, don't sell it.
    #TODO: Need to double check the BDay logic holds given the timezone naive datetime objects. Since trading hours extend into midnight China time, this might affect the logic here.
    if (position_series.bought_datetime + BDay(2) > today):
        pass
    
    # If we're past the fireSaleAge, and the CurrPrice is below the fireSalePrice or the CostBasis is higher than the CurrPrice, then place a sell order
    elif (
        position_series.bought_datetime + BDay(fireSaleAge) < today 
        and (
            fireSalePrice>CurrPrice
            or CostBasis>CurrPrice
            )
        ):
        SellPrice = float(ib.make_div_by_05(.95*CurrPrice, buy=False))
        print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
        orderid = app.submit_order(resolved_ibcontract=resolved_ibcontract, action="SELL", shares=StockShares, price=SellPrice, order_type="LMT", orderid=None, tif="GTC")
    else:
        print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
        orderid = app.submit_order(resolved_ibcontract=resolved_ibcontract, action="SELL", shares=StockShares, price=SellPrice, order_type="LMT", orderid=None, tif="GTC")
        
    return orderid

In [17]:
positions = app.get_current_positions()
positions

,account,position,avg_cost,comboLegs,comboLegsDescrip,conId,currency,deltaNeutralContract,exchange,includeExpired,...,multiplier,primaryExchange,right,secId,secIdType,secType,strike,symbol,tradingClass,bought_datetime
0,U9509931,90.0,6.261111,None,,324062330,USD,None,NASDAQ,False,...,,,,,,STK,0.0,TBIO,NMS,2018-11-14 00:04:37
1,U9509931,103.0,19.359709,None,,327422071,USD,None,NASDAQ,False,...,,,,,,STK,0.0,LQDA,SCM,2018-11-14 01:52:41
2,U9509931,264.0,7.555000,None,,322592376,USD,None,NASDAQ,False,...,,,,,,STK,0.0,MGTA,NMS,2018-11-14 01:48:38
3,U9509931,232.0,8.600000,None,,329820282,USD,None,NASDAQ,False,...,,,,,,STK,0.0,SIC,SCM,2018-11-14 01:46:32
4,U9509931,243.0,8.205000,None,,294729588,USD,None,NASDAQ,False,...,,,,,,STK,0.0,ALNA,NMS,2018-11-13 23:53:21
5,U9509931,110.0,18.059091,None,,319099329,USD,None,NASDAQ,False,...,,,,,,STK,0.0,KNSA,NMS,2018-11-14 00:08:04
6,U9509931,200.0,5.455000,None,,310624804,USD,None,NASDAQ,False,...,,,,,,STK,0.0,LAZY,SCM,2018-11-14 00:04:11
7,U9509931,128244.0,1.352587,None,,37928772,SGD,None,,False,...,,,,,,CASH,0.0,USD,USD.SGD,NaT


In [27]:
i = 0
rebalance_sell_order(positions.loc[i])

Getting full contract details from the server... 
got multiple contracts using first one
324062330,TBIO,STK,,0.0,,,SMART,NASDAQ,USD,TBIO,NMS,False,,combo:


,ask_price,ask_size,bid_price,bid_size,last_trade_price,last_trade_size
2018-11-19 01:52:54,-1.0,0.0,-1.0,0.0,NaN,0.0


nan
Selling TBIO, 	shares: 90.0, 	price: 6.35
App is connected?: True
Getting orderid from IB
Using order id of 18
Placed order, orderid is 18


18

In [ ]:
def rebalance_all_sell_orders():

    positions = app.get_current_positions()

    # Get list of open sell orders
    open_sell_orders = get_open_sell_orders()

    # Review if limit sell order exists for every position    
    for i, position in positions.iterrows():

        # Check if stock has a existing limit sell order. If not, add one
        # TODO: Also check that the position is not 'USD'
        if not open_sell_orders(stock):
            rebalance_sell_order(position)

In [ ]:
## lets get positions
positions_list = app.get_current_positions()
print(positions_list)

## get the account name from the position
## normally you would know your account name
accountName = positions_list.account[0]

## and accounting information
accounting_values = app.get_accounting_values(accountName)
print(accounting_values)

## these values are cached
## if we ask again in more than 5 minutes it will update everything
accounting_updates = app.get_accounting_updates(accountName)
print(accounting_updates)

# TODO: solve this error below
'''
  File "/Users/chingaling/Documents/Algo Trading/BroCodePC/IB_API_tutorial.py", line 380, in updateAccountTime
    self._my_accounts[accountName].put(data)
NameError: name 'accountName' is not defined
'''

In [ ]:
def positions_list_pd(positions_list):
    columns = ['account', 'contract', 'position', 'avg_cost']
    positions_list_pd = pd.DataFrame(positions_list, columns=columns)
    return positions_list_pd

In [ ]:
accounting_values

In [ ]:
accounting_updates[0]

In [ ]:
# Re-balancing method from 30_stock algo every 10 minutes. Buy order cycles through list of 100 stock selected each day.

def my_rebalance(context, data):
    buyFactor=.99
    SellFactor=1.01
    cash=context.portfolio.cash

    cancel_open_buy_orders(context, data)

    # Order sell at profit target in hope that somebody actually buys it
    for stock in context.portfolio.positions:
        if not get_open_orders(stock):
            StockShares = context.portfolio.positions[stock].amount
            CurrPrice = float(data.current([stock], 'price'))
            CostBasis = float(context.portfolio.positions[stock].cost_basis)
            SellPrice = float(make_div_by_05(CostBasis*SellFactor, buy=False))
            
            
            if np.isnan(SellPrice) :
                pass # probably best to wait until nan goes away
            elif (stock in context.age and context.age[stock] == 1) :
                pass
            elif (
                stock in context.age 
                and context.MyFireSaleAge<=context.age[stock] 
                and (
                    context.MyFireSalePrice>CurrPrice
                    or CostBasis>CurrPrice
                )
            ):
                if (stock in context.age and context.age[stock] < 2) :
                    pass
                elif stock not in context.age:
                    context.age[stock] = 1
                else:
                    SellPrice = float(make_div_by_05(.95*CurrPrice, buy=False))
                    print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
                    order(stock, -StockShares,
                        style=LimitOrder(SellPrice)
                    )
            else:
                if (stock in context.age and context.age[stock] < 2) :
                    pass
                elif stock not in context.age:
                    context.age[stock] = 1
                else:
                    print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
                    order(stock, -StockShares,
                        style=LimitOrder(SellPrice)
                    )

    WeightThisBuyOrder=float(1.00/context.maxBuyOrdersAtOnce)
    for ThisBuyOrder in range(context.maxBuyOrdersAtOnce):
        stock = next(context.MyCandidate)
        PH = data.history([stock], 'price', 20, '1d')
        PH_Avg = float(PH.mean())
        CurrPrice = float(data.current([stock], 'price'))
        if np.isnan(CurrPrice):
            pass # probably best to wait until nan goes away
        else:
            if CurrPrice > float(1.25*PH_Avg):
                BuyPrice=float(CurrPrice)
            else:
                BuyPrice=float(CurrPrice*buyFactor)
            BuyPrice=float(make_div_by_05(BuyPrice, buy=True))
            StockShares = int(WeightThisBuyOrder*cash/BuyPrice)
            #print("Buying {}, \tshares: {}, \tprice: {}".format(stock, StockShares, BuyPrice))
            order(stock, StockShares,
                style=LimitOrder(BuyPrice)
            )

## Lesson 1: Create a connection with IB API
https://qoppac.blogspot.com/2017/03/interactive-brokers-native-python-api.html

## Lesson 2: Get historic data
https://qoppac.blogspot.com/2017/03/historic-data-from-native-ib-pyhon-api.html

## Lesson 3: Get streaming data
https://qoppac.blogspot.com/2017/03/streaming-market-data-from-native.html

## Lesson 4: Creating orders
https://qoppac.blogspot.com/2017/03/placing-orders-in-native-python-ib-api.html

## Lesson 5: Getting portfolio positions and account info
https://qoppac.blogspot.com/2017/03/getting-position-and-accounting-data.html